In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [11]:
# CONSTANTS
tubL, tubW, tubH = 1.20, 0.60, 0.40 # meters
molWaterInM3 = 55493.9 # mol / meter^3
humanCP = 62.97 # J / mol K

In [35]:
# VARIABLES
countL, countW, countH = 12, 6, 4 # dimensionless
waterStartingTemp = 37.0 # deg celsius
ambientTemp = 20.0 # deg celsius
bathWallThermalDiffusivity = 22e-6 # cast iron thermal diffusivity
bathWallThermalConductivity = 55 # cast iron W/m K
bathWallSpecificHeat = 500 # J / kg K
bathWallThickness = 5e-3 # cast iron thickness, m
bathWallDensity = 7170 # cast iron density, kg/m^3
kMetal = 0.20

dt = 0.1 # seconds

In [36]:
# DERIVED
distL, distW, distH = tubL/countL, tubW/countW, tubH/countH
saL, saW, saH = distW*distH, distL*distH, distL*distW
wallWtL = saL * (bathWallThickness*bathWallDensity) # kg
wallWtW = saW * (bathWallThickness*bathWallDensity) # kg
wallWtH = saH * (bathWallThickness*bathWallDensity) # kg
#kMetalL = bathWallThermalConductivity / ((bathWallThickness ** 2) * bathWallSpecificHeat * bathWallDensity) # SA * TC / TH * [J/K]
#print(saL, kMetalL)

In [37]:
def getWaterDiffusivity(temp):
    lmda = 0.561 + 0.002 * temp + 0.00000962 * (temp ** 2)
    cp = 75.98 - 0.061 * temp + 0.002 * (temp ** 2) - 0.00001755 * (temp ** 3)
    rho = 999.86 + 0.058 * temp - 0.008 * (temp ** 2) + 0.0000397 * (temp ** 3)
    return lmda / (cp * rho)

def getSubsectionVolumeMols():
    return molWaterInM3 * (tubL/countL) * (tubW/countW) * (tubH/countH)

def getNewTempArray():
    return waterStartingTemp*np.ones((countL+1, countW+1, countH+1))

def getTubSafe(tub, l, w, h):
    if l < 0 or l > countL: return waterStartingTemp
    if w < 0 or w > countW: return waterStartingTemp
    if h < 0 or h > countH: return waterStartingTemp
    return tub[l][w][h]

def getPointDiffusivity(tub, l, w, h):
    if l == 0 or l == countL: return bathWallThermalDiffusivity
    if w == 0 or w == countW: return bathWallThermalDiffusivity
    if h == 0 or h == countH: return bathWallThermalDiffusivity
    return getWaterDiffusivity(tub[l][w][h])

def getCellLagrangian(tub, l, w, h):
    ddL = float(getTubSafe(tub, l+1, w, h) + getTubSafe(tub, l-1, w, h) - 2 * getTubSafe(tub, l, w, h)) / (distL ** 2)
    ddW = float(getTubSafe(tub, l, w+1, h) + getTubSafe(tub, l, w-1, h) - 2 * getTubSafe(tub, l, w, h)) / (distW ** 2)
    ddH = float(getTubSafe(tub, l, w, h+1) + getTubSafe(tub, l, w, h-1) - 2 * getTubSafe(tub, l, w, h)) / (distH ** 2)
    return ddL + ddW + ddH

def cellStep(tub):
    diff = np.copy(tub)
    newTub = np.copy(tub)
    for l in range(0, countL+1):
        for w in range(0, countW+1):
            for h in range(0, countH+1):
                wd = getPointDiffusivity(tub, l, w, h)
                dd = getCellLagrangian(tub, l, w, h)
                mm = getSubsectionVolumeMols()
                diff[l][w][h] = dd#dt*mm*wd*dd
                newTub[l][w][h] += dt*mm*wd*dd
                if l == 0 or w == 0 or l == countL or w == countW or h == countH:
                    newTub[l][w][h] += -0.20*dt*(newTub[l][w][h]-ambientTemp)
    return newTub

In [40]:
arr = getNewTempArray();
arr[1][1][1] = 90

for i in range(0, 600):
    arr = cellStep(arr)
print(arr)

[[[ 30.31695007  28.95851491  28.68224294  28.82765577  30.09675595]
  [ 29.5895197   27.85440722  27.40018862  27.46403673  28.81221757]
  [ 29.58228348  27.79401508  27.27365105  27.27003552  28.59062364]
  [ 29.6040731   27.81502112  27.2793775   27.25436691  28.56145039]
  [ 29.58068981  27.79052355  27.26965758  27.26711539  28.58963022]
  [ 29.58762971  27.85030152  27.39549621  27.46059564  28.81102554]
  [ 30.31614306  28.95689471  28.6804006   28.82627802  30.0961977 ]]

 [[ 29.59206156  27.85835661  27.40401743  27.46658178  28.81315431]
  [ 31.79299054  29.47350873  28.45061177  27.91553347  27.43925313]
  [ 32.4943383   30.20906678  28.98975811  28.13872489  27.24125304]
  [ 32.66837748  30.42242044  29.15860395  28.21854867  27.22478625]
  [ 32.48858375  30.19435831  28.97288431  28.12663956  27.23837972]
  [ 31.78629343  29.45651816  28.43113218  27.90155294  27.43586663]
  [ 29.58993356  27.85378711  27.39880289  27.46274992  28.81179994]]

 [[ 29.597699    27.81773144  

In [ ]:
print(saL, saW, saH)
print(wallWtL, wallWtW, wallWtH)